In [1]:
%%bash
# If your project has a 'requirements.txt' file, we'll install it here apart from blacklisted packages that interfere with Deepnote (see above).
cd /work
if test -f requirements.txt
  then
    sed -i '/jedi/d;/jupyter/d;' ./requirements.txt
    # Deepnote's notebook 6.5.5 used to require pyzmq<25.0.0,
    # but this seems to no longer be the case.
    # Here we install nbconvert manually, see https://github.com/RussTedrake/manipulation/issues/481
    pip install -r ./requirements.txt nbconvert
  else echo "There's no requirements.txt, so nothing to install. This is the case with most projects."
fi

bash: line 2: cd: /work: No such file or directory


Looking in indexes: https://pypi.org/simple, https://drake-packages.csail.mit.edu/whl/nightly
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/744.6 kB ? eta -:--:--━━━━━━━━━━━ 744.6/744.6 kB 9.3 MB/s  0:00:00
  Attempting uninstall: manipulation
    Found existing installation: manipulation 2025.10.6
    Uninstalling manipulation-2025.10.6:
      Successfully uninstalled manipulation-2025.10.6


# Catching a Falling Object

In this notebook, we will learn how to catch an object that is falling with a Kuka iiwa-7. This task is difficult because of the timing components. You will be responsible for calculating the rate at which an object is falling using several Intel Realsense cameras and then calculating the expected position of the object at a certain point in time, where you will catch the object.


**Learning Objectives:**
1. Implement a system that is able to accurately prediction the falling location of a system above.
2. Implement control (probably force control) to catch a falling object without it hitting the ground.

**What you'll build:** A complete system with Kuka and Realsense cameras that is able to estimate the falling position of an object and catch the object.

---

In [ ]:
# Cut down on this once you decide what libraries you actually want/need.

import time
from typing import Literal

import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BasicVector,
    ContactVisualizer,
    ContactVisualizerParams,
    Context,
    DiagramBuilder,
    FixedOffsetFrame,
    InputPort,
    InverseKinematics,
    JacobianWrtVariable,
    LeafSystem,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    ModelInstanceIndex,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    OutputPort,
    Parser,
    PiecewisePolynomial,
    PrismaticJoint,
    RevoluteJoint,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Solve,
    SpatialInertia,
    StartMeshcat,
    TrajectorySource,
    UnitInertia,
)

from manipulation import running_as_notebook
from manipulation.scenarios import AddMultibodyTriad, SetColor
from manipulation.station import AppendDirectives, LoadScenario, MakeHardwareStation
from manipulation.utils import ConfigureParser, FindResource, RenderDiagram